# Aprendizaje Supervisado - Ejemplo Titanic

Este notebook realiza los siguientes pasos:
1. Carga los datos de entrenamiento y prueba del Titanic.
2. Preprocesa los datos para convertirlos a un formato numérico y manejar valores faltantes.
3. Divide los datos de entrenamiento para poder evaluar el modelo.
4. Entrena un modelo de clasificación (Random Forest).
5. Evalúa el rendimiento del modelo usando la métrica de precisión (accuracy).
6. Genera un archivo de predicciones para la competencia de Kaggle.

In [ ]:
# --- 0. Importar librerías necesarias ---
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import os

## 1. Definición de rutas y variables

Definimos las rutas de los archivos y las columnas que usaremos en el modelo.

In [ ]:
# Si ejecutas en Google Colab o Jupyter, adapta las rutas a tu entorno
TRAIN_FILE_PATH = "titanic/train.csv"
TEST_FILE_PATH = "titanic/test.csv"
SUBMISSION_FILE_PATH = "submission.csv"

FEATURES = ['Pclass', 'Sex', 'Age']
TARGET = 'Survived'

## 2. Cargar los datos

In [ ]:
print("Cargando datos...")

try:
    train_df = pd.read_csv(TRAIN_FILE_PATH)
    test_df = pd.read_csv(TEST_FILE_PATH)
    print("Datos cargados exitosamente.")
except FileNotFoundError as e:
    print("Error: No se encontró el archivo.")
    print(f"Ruta esperada: {e}")

## 3. Preprocesamiento de datos
- Convertimos la columna `Sex` a numérico.
- Rellenamos valores faltantes de `Age` con la **mediana** del conjunto de entrenamiento.

In [ ]:
# Copias de seguridad para evitar modificaciones no deseadas
train_processed = train_df.copy()
test_processed = test_df.copy()

# Mapear sexos a números
sex_mapping = {'male': 0, 'female': 1}
train_processed['Sex'] = train_processed['Sex'].map(sex_mapping)
test_processed['Sex'] = test_processed['Sex'].map(sex_mapping)

# Rellenar NaN en Age con la mediana del train
median_age = train_processed['Age'].median()
print(f"Mediana de edad usada: {median_age:.2f}")

train_processed['Age'] = train_processed['Age'].fillna(median_age)
test_processed['Age'] = test_processed['Age'].fillna(median_age)

## 4. Entrenamiento y evaluación del modelo

In [ ]:
# Dividir datos en train y validación
X = train_processed[FEATURES]
y = train_processed[TARGET]

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Datos de entrenamiento: {len(X_train)}, Validación: {len(X_val)}")

# Entrenar modelo
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluación
preds = clf.predict(X_val)
accuracy = accuracy_score(y_val, preds)

print(f"Precisión en validación: {accuracy:.2%}")

## 5. Generar archivo de submission para Kaggle

In [ ]:
# Predicciones en test
X_test = test_processed[FEATURES]
test_predictions = clf.predict(X_test)

# Crear archivo de salida
output = pd.DataFrame({
    'PassengerId': test_processed.PassengerId,
    'Survived': test_predictions
})

output.to_csv(SUBMISSION_FILE_PATH, index=False)
print(f"Archivo guardado en {SUBMISSION_FILE_PATH}")

## ✅ Conclusión
Hemos construido un pipeline básico de **Aprendizaje Supervisado** con Random Forest
para el dataset del Titanic.  
Este notebook es fácilmente ampliable:
- Se pueden agregar más variables (Fare, SibSp, Embarked, etc.)
- Se pueden probar distintos algoritmos de ML.
- Se pueden ajustar hiperparámetros con GridSearchCV.